<a href="https://colab.research.google.com/github/Domefy/AI/blob/main/%E2%80%9Csovits4_0%E4%B8%80%E9%94%AE%E8%AE%AD%E7%BB%83_%E6%8E%A8%E7%90%86%E8%84%9A%E6%9C%AC_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 查看显卡
!nvidia-smi

Sun Apr 30 13:19:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
#@title 克隆的github仓库
!git clone https://github.com/svc-develop-team/so-vits-svc.git -b 4.0


Cloning into 'so-vits-svc'...
remote: Enumerating objects: 1326, done.
remote: Total 1326 (delta 0), reused 0 (delta 0), pack-reused 1326
Receiving objects: 100% (1326/1326), 8.46 MiB | 18.39 MiB/s, done.
Resolving deltas: 100% (726/726), done.


In [2]:
#@title 安装依赖
%cd /content/so-vits-svc
!pip install pyworld praat-parselmouth fairseq


/content/so-vits-svc
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 94.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.7/272.7 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 16.8 MB/s eta 0:00:00
  

In [3]:
#@title 下载必要模型文件
# 源仓库地址：[contentvec](https://github.com/auspicious3000/contentvec)
# 模型原下载链接：[checkpoint_best_legacy_500.pt](https://ibm.box.com/s/z1wgl1stco8ffooyatzdwsqn2psd9lrr)
# 由于源网盘无法提供http直链，根据mit协议，对模型进行二次分发，提供下载直链
!wget -P hubert/ https://huggingface.co/innnky/contentvec/resolve/main/checkpoint_best_legacy_500.pt

!wget -P logs/44k/ https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/D_0.pth

!wget -P logs/44k/ https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/G_0.pth


--2023-04-30 13:24:27--  https://huggingface.co/innnky/contentvec/resolve/main/checkpoint_best_legacy_500.pt
Resolving huggingface.co (huggingface.co)... 18.155.68.38, 18.155.68.116, 18.155.68.44, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/0a/5f/0a5f89ff9058782f4b7a207d38e0914d63f6f55e8fc1fad0c3bf68b546039f2d/60d936ec5a566776fc392e69ad8b630d14eb588111233fe313436e200a7b187b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27checkpoint_best_legacy_500.pt%3B+filename%3D%22checkpoint_best_legacy_500.pt%22%3B&Expires=1683120267&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzBhLzVmLzBhNWY4OWZmOTA1ODc4MmY0YjdhMjA3ZDM4ZTA5MTRkNjNmNmY1NWU4ZmMxZmFkMGMzYmY2OGI1NDYwMzlmMmQvNjBkOTM2ZWM1YTU2Njc3NmZjMzkyZTY5YWQ4YjYzMGQxNGViNTg4MTExMjMzZmUzMTM0MzZlMjAwYTdiMTg3Yj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb

In [4]:
!wget -P preModles https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/D_0.pth

!wget -P preModles https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/G_0.pth

--2023-04-30 13:26:25--  https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/D_0.pth
Resolving huggingface.co (huggingface.co)... 18.155.68.116, 18.155.68.121, 18.155.68.38, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.116|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/c7/21/c7214c5855c8c13ab3ad357ebc651593e9ca56b492469e678bdbda597a830f99/635be5c3409aaf3eec4135a1f5a771595683f3a6461ffc5bdea43441e50269a9?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27D_0.pth%3B+filename%3D%22D_0.pth%22%3B&Expires=1683119714&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2M3LzIxL2M3MjE0YzU4NTVjOGMxM2FiM2FkMzU3ZWJjNjUxNTkzZTljYTU2YjQ5MjQ2OWU2NzhiZGJkYTU5N2E4MzBmOTkvNjM1YmU1YzM0MDlhYWYzZWVjNDEzNWExZjVhNzcxNTk1NjgzZjNhNjQ2MWZmYzViZGVhNDM0NDFlNTAyNjlhOT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFR

# 数据集预处理

将wav数据修改  https://huggingface.co/cenj/xunlian/tree/main



In [5]:
!wget -P dataset_raw https://huggingface.co/cenj/xunlian/resolve/main/mxd.zip

!wget -P dataset_raw https://huggingface.co/cenj/xunlian/resolve/main/zjl.zip



--2023-04-30 14:34:49--  https://huggingface.co/cenj/xunlian/resolve/main/mxd.zip
Resolving huggingface.co (huggingface.co)... 18.155.68.121, 18.155.68.38, 18.155.68.116, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.121|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/d5/74/d574e47c507ff12aa3ab3e35994cd107758322f288e41b9abe7f612e676c664d/954bbc55b60129720421acba747bebc8c8045538d75826ba52e2077d6320c899?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27mxd.zip%3B+filename%3D%22mxd.zip%22%3B&response-content-type=application%2Fzip&Expires=1683124490&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2Q1Lzc0L2Q1NzRlNDdjNTA3ZmYxMmFhM2FiM2UzNTk5NGNkMTA3NzU4MzIyZjI4OGU0MWI5YWJlN2Y2MTJlNjc2YzY2NGQvOTU0YmJjNTViNjAxMjk3MjA0MjFhY2JhNzQ3YmViYzhjODA0NTUzOGQ3NTgyNmJhNTJlMjA3N2Q2MzIwYzg5OT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9uc2UtY29udGVudC10eXBlPSoiLC

解压缩mxd.zip和zjl.zip

In [ ]:


!unzip dataset_raw/mxd.zip -d dataset_raw/

!unzip dataset_raw/zjl.zip -d dataset_raw/

In [ ]:
!mkdir dataset_raw/mxd
!mkdir dataset_raw/zjl

In [11]:
!rm -r dataset_raw/mxd

!rm -r dataset_raw/zjl

In [13]:
!cp /content/config.json dataset_raw/

In [ ]:
#@title 加载Google云端硬盘
#@markdown 加载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


该处理脚本可以一次性预处理多个说话人，并且生成多说话人filelists以及对应的配置文件

只需要将你的数据集按照以下文件结构放到dataset_raw目录下


```
dataset_raw
├───speaker0
│   ├───xxx1-xxx1.wav
│   ├───...
│   └───Lxx-0xx8.wav
└───speaker1
    ├───xx2-0xxx2.wav
    ├───...
    └───xxx7-xxx007.wav

```




In [ ]:
#@title 从谷歌云盘加载打包好的数据集进行预处理
#@markdown **sovits3.0的数据集不再需要特定的文件结构，将数据集的所有wav文件放在同一个文件夹下压缩为zip后上传至谷歌云盘即可，该处理脚本可以一次性预处理多个数据集，处理多个数据集时请依次解压每一个数据集**

#@markdown 数据集名称（**人物的英文/拼音名**，与建数据文件夹时统一；不带zip。）
DATASETNAME = "kiritan"  #@param {type:"string"}
#@markdown 压缩包路径（谷歌盘路径，传到dataset的就不改这个，没有dataset文件夹就新建一个）
ZIP_PATH = "/content/drive/MyDrive/dataset/"  #@param {type:"string"}
ZIP_NAME = ZIP_PATH + DATASETNAME

!unzip -d /content/so-vits-svc/dataset_raw {ZIP_NAME}.zip

In [14]:
#@title 重采样到44100hz
!python resample.py

./dataset_raw/zjl
877it [01:10, 12.48it/s]
./dataset_raw/mxd
493it [00:42, 11.52it/s]


In [15]:
#@title 划分训练集 生成配置文件
!python preprocess_flist_config.py

  0% 0/2 [00:00<?, ?it/s]skip too short audio: ./dataset/44k/zjl/zjl_1628.wav
skip too short audio: ./dataset/44k/zjl/zjl_1317.wav
skip too short audio: ./dataset/44k/zjl/zjl_1319.wav
skip too short audio: ./dataset/44k/zjl/zjl_1494.wav
skip too short audio: ./dataset/44k/zjl/zjl_1247.wav
skip too short audio: ./dataset/44k/zjl/zjl_1158.wav
skip too short audio: ./dataset/44k/zjl/zjl_1430.wav
skip too short audio: ./dataset/44k/zjl/zjl_1032.wav
skip too short audio: ./dataset/44k/zjl/zjl_1474.wav
skip too short audio: ./dataset/44k/zjl/zjl_1344.wav
skip too short audio: ./dataset/44k/mxd/mxd_1204.wav
skip too short audio: ./dataset/44k/mxd/mxd_1441.wav
100% 2/2 [00:00<00:00, 29.73it/s]
Writing ./filelists/train.txt
100% 1354/1354 [00:00<00:00, 1127474.21it/s]
Writing ./filelists/val.txt
100% 4/4 [00:00<00:00, 128070.35it/s]
Writing configs/config.json


In [ ]:
#@title 生成hubert和f0
!python preprocess_hubert_f0.py

[1370]
Loading hubert for content...
load model(s) from hubert/checkpoint_best_legacy_500.pt
2023-04-30 15:25:22.561042: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-04-30 15:25:24.095453: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:jaxlib.mlir._mlir_libs:Initializing MLIR with module: _site_initialize_0
DEBUG:jaxlib.mlir._mlir_libs:Registering dialects from i

In [22]:
!sudo apt-get install build-essential python-dev python3-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'python-dev-is-python2' instead of 'python-dev'
python3-dev is already the newest version (3.8.2-0ubuntu2).
python3-dev set to manually installed.
build-essential is already the newest version (12.8ubuntu1.1).
The following additional packages will be installed:
  libpython2-dev libpython2-stdlib libpython2.7 libpython2.7-dev
  python-is-python2 python2 python2-dev python2-minimal python2.7-dev
Suggested packages:
  python2-doc python-tk
The following NEW packages will be installed:
  libpython2-dev libpython2-stdlib libpython2.7 libpython2.7-dev
  python-dev-is-python2 python-is-python2 python2 python2-dev python2-minimal
  python2.7-dev
0 upgraded, 10 newly installed, 0 to remove and 24 not upgraded.
Need to get 3,870 kB of archives.
After this operation, 18.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 python2-minim

In [23]:
!pip install --no-cache-dir numpy==1.20.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 201.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: Could not build wheels for numpy, which is required to install pyproject.toml-based projects


In [ ]:
!pip install -r requirements.txt

In [25]:
!pip install git+https://github.com/JeremyCCHsu/Python-Wrapper-for-World-Vocoder.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/JeremyCCHsu/Python-Wrapper-for-World-Vocoder.git to /tmp/pip-req-build-zval_8zp
  Running command git clone --filter=blob:none --quiet https://github.com/JeremyCCHsu/Python-Wrapper-for-World-Vocoder.git /tmp/pip-req-build-zval_8zp
  Resolved https://github.com/JeremyCCHsu/Python-Wrapper-for-World-Vocoder.git to commit c999deaf9add59ebdaff4979408a669518bba389
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
#@title 至此 数据集预处理制作完毕，将数据集和相关文件保存到谷歌云盘的dataset文件夹下，方便下次训练使用
#压缩dataset文件夹
!zip -r dataset.zip /content/so-vits-svc/dataset
#@markdown 自定义备份到谷歌云盘的dataset文件夹下的数据集文件夹名，避免混淆
dataset_name_drive = "kiritan_preprocessed"  #@param {type:"string"}
DATASET_PATH_DRIVE = "/content/drive/MyDrive/dataset/" + dataset_name_drive
!mkdir -p {DATASET_PATH_DRIVE}

!cp /content/so-vits-svc/dataset.zip "{DATASET_PATH_DRIVE}"
!cp configs/config.json "{DATASET_PATH_DRIVE}"
!cp filelists/train.txt "{DATASET_PATH_DRIVE}"
!cp filelists/val.txt "{DATASET_PATH_DRIVE}"

  adding: content/so-vits-svc/dataset/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/kiritan/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000049.wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000020.wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000027.wav.f0.npy (deflated 46%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000007.wav.f0.npy (deflated 30%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000026.wav.f0.npy (deflated 21%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000000.wav (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000039.wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000035.wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000024.wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000004.wav (deflated 5%)
  adding: content

In [ ]:
#@title 已经预处理过数据集的话，就可以跳过预处理部分 直接从云盘解压处理过的数据 以及配置文件
#@markdown 从谷歌云盘加载预处理过的数据集，文件夹名和你备份的时候输入的一样
back_up_name = "kiritan_preprocessed"  #@param {type:"string"}
BACK_UP_DATASET_PATH = "/content/drive/MyDrive/dataset/" + back_up_name
!unzip {BACK_UP_DATASET_PATH}/dataset.zip -d /
!cp {BACK_UP_DATASET_PATH}/config.json /content/so-vits-svc/configs/config.json 
!cp {BACK_UP_DATASET_PATH}/val.txt filelists/val.txt
!cp {BACK_UP_DATASET_PATH}/train.txt filelists/train.txt


# 拷贝云盘上保存的记录点
# !cp /content/drive/MyDrive/G_800.pth logs/48k/
# !cp /content/drive/MyDrive/D_800.pth logs/48k/

Archive:  /content/drive/MyDrive/dataset/kiritan_preprocessed/dataset.zip
replace /content/so-vits-svc/dataset/44k/kiritan/000049.wav.soft.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000049.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000020.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000027.wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000007.wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000026.wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000000.wav  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000039.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000035.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000024.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000004.wav  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000029.wav  
  inflating: /conte

# 训练

In [ ]:
#@title  选择是否将训练后的模型保存到谷歌云盘 和 是否使用预模型

#@markdown **将训练后的模型文件保存到谷歌云盘，勾选后，恢复训练时也需要勾选并执行**
Save_to_drive = True #@param {type:"boolean"}
if Save_to_drive:
  !rm -rf /content/so-vits-svc/logs/44k
  !mkdir -p /content/drive/MyDrive/44k
  !ln -s /content/drive/MyDrive/44k /content/so-vits-svc/logs/44k

#@markdown **首次训练下载预模型 之后继续训练则使用自己保存的记录点，无需再下载**

#@markdown **使用预模型，下面打钩自动下载并启用**
pre_pth = True #@param {type:"boolean"}
if pre_pth:
  !wget -P logs/44k/ https://huggingface.co/innnky/sovits_pretrained/resolve/main/sovits4/G_0.pth
  !wget -P logs/44k/ https://huggingface.co/innnky/sovits_pretrained/resolve/main/sovits4/D_0.pth


--2023-02-20 06:05:15--  https://huggingface.co/innnky/sovits_pretrained/resolve/main/sovits4/G_0.pth
Resolving huggingface.co (huggingface.co)... 3.231.67.228, 54.235.118.239, 2600:1f18:147f:e800:671:b733:ecf3:a585, ...
Connecting to huggingface.co (huggingface.co)|3.231.67.228|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b8/68/b8680b5490e63bf5f140618c3ee2e5ccd0731956a87ab87a228c08398cd8e03c/20a327c54e5731bed377bd38404bc32ab98e66a1b2777b0af4cc034d4d6914b0?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27G_0.pth%3B+filename%3D%22G_0.pth%22%3B&Expires=1677132316&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2I4LzY4L2I4NjgwYjU0OTBlNjNiZjVmMTQwNjE4YzNlZTJlNWNjZDA3MzE5NTZhODdhYjg3YTIyOGMwODM5OGNkOGUwM2MvMjBhMzI3YzU0ZTU3MzFiZWQzNzdiZDM4NDA0YmMzMmFiOThlNjZhMWIyNzc3YjBhZjRjYzAzNGQ0ZDY5MTRiMD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc

In [ ]:
#@title  开始训练

#@markdown **开始训练**

#@markdown **启用tensorboard可视化数据**
tensorboard_on = True #@param {type:"boolean"}
if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/44k

!python train.py -c configs/config.json -m 44k


In [ ]:
#@title 手动将训练后的模型文件备份到谷歌云盘
#@markdown 需要自己查看/content/so-vits-svc/logs/44k/文件夹下模型的文件名，手动修改下方命令末尾的文件名
!mv /content/so-vits-svc/logs/44k/G_1000.pth /content/drive/MyDrive
!mv /content/so-vits-svc/logs/44k/D_1000.pth /content/drive/MyDrive

# 推理

In [ ]:
#@title 合成音频（推理）
#@markdown 需要将音频上传到so-vits-svc/raw 文件夹下, 然后设置模型路径、配置文件路径、合成的音频名称

!python inference_main.py -m "logs/44k/G_1600.pth" -c "configs/config.json" -n "君の知らない物語-src.wav" -s kiritan


load model(s) from hubert/checkpoint_best_legacy_500.pt
INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
INFO:fairseq.tasks.hubert_pretraining:current directory is /content/so-vits-svc
INFO:fairseq.tasks.hubert_pretraining:HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
INFO:fairseq.models.hubert.hubert:HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 0.0, 'encoder_layerdro